# Assignment 16

Below is a class called `MotionDetector` that takes an MP4 filename as an argument upon class instantiation and then is intended to perform several manipulations of the underlying video. Upon class instantiation, the member function `create_grayscale_video_matrix` extracts each frame from from the video and converts the color images to grayscale. Each frame is represented by and integer number between 0 and 255 in a  array where  is the number of pixel in the height and  is the number of pixes along the width of the frame. Each of these arrays are "flattened", i.e. turned into a one-dimensional array, and then "stacked" into a matrix where every row represents the flattened array from each frame in the movie.

Your assignment is to implement the function `compute_low_rank_images`. The function should compute the SVD of the class attribute `images_matrix`, and modify `images_matrix` to contain the "low rank" matrix. The keyword argument "rank" refers to the number of singular values that should be used when recontructing the images_matrix. To be clear, your SVD should return three matrices, $\mathbf{U}, \boldsymbol{\Sigma}, \mathbf{V}$, then you should modify these matrices to exclude all singular values greater than rank and then multiply the matrices back together and assign it to the class attribute `images_matrix`.

After you do this, you should be able to call `compute_low_rank_images` (with a small value for rank), followed by `create_movie` and `display_movie`. What do you see?

The original video is displayed for reference currently. Please comment this cell out (and any other cells you used for debugging) leaving only the class definition.

In [1]:
import cv2
import numpy as np
import os
from IPython.display import HTML
import matplotlib.pyplot as plt
import shutil

class MotionDetector():

    def __init__(self, filename):
        self.filename = filename
        self.image_directory_name = '.frames'
        self.create_grayscale_video_matrix()

    def __del__(self):
        if os.path.isdir(self.image_directory_name):
            shutil.rmtree(self.image_directory_name)

    def create_grayscale_video_matrix(self):
        vidcap = cv2.VideoCapture(self.filename)
        success, image = vidcap.read()
        count = 0
        while success:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            if count == 0:
                images = gray_image.flatten().reshape((1, -1))
            images = np.append(images, [gray_image.flatten()], axis=0)
            success, image = vidcap.read()
            count += 1
        self.image_shape = gray_image.shape
        self.images_matrix = images.astype('int')
        self.number_of_frames = count

        return
    
    def write_images(self, basename=None):
        if basename is None:
            basename = '{}_frame'.format(self.filename.split('.')[0])
        if not os.path.isdir(self.image_directory_name):
            os.mkdir(self.image_directory_name)
        for i, img in enumerate(self.images_matrix):
            img = img.reshape(self.image_shape)
            filepath = os.path.join(self.image_directory_name, '{}{}.png'.format(basename, i))
            if os.path.isfile(filepath):
                os.remove(filepath)
            cv2.imwrite(filepath, img)

    def compute_low_rank_images(self, rank=2):
        U, S, VT = np.linalg.svd(self.images_matrix, full_matrices=False)
        
        S_reduced = np.diag(S[:rank])
        U_reduced = U[:, :rank]
        VT_reduced = VT[:rank, :]
        print(np.shape(self.images_matrix))
        self.images_matrix = np.dot(U_reduced, np.dot(S_reduced, VT_reduced)).astype('int')
        

        return

    def create_image_difference(self, rank=2):
        original_images = self.images_matrix.copy()
        self.compute_low_rank_images(rank=rank)
        self.images_matrix -= original_images

    def create_movie(self, basename=None):
        if basename is None:
            basename = '{}_modified'.format(self.filename.split('.')[0])
        self.write_images(basename)
        os.system('ffmpeg -y -i {}/{}%d.png -vcodec libx264 {}.mp4'.format(self.image_directory_name, basename, basename))

    def display_original_movie(self):
        video_path = self.filename
        return HTML("""
        <video width="600" height="400" controls="">
        <source src="{0}">
        </video>
        """.format(video_path))

    def display_movie(self, basename=None):
        if basename is None:
            basename = '{}_modified'.format(self.filename.split('.')[0])
        video_path = '{}.mp4'.format(basename)
        return HTML("""
        <video width="600" height="400" controls="">
        <source src="{0}">
        </video>
        """.format(video_path))

    def display_image_plot(self):
        plt.figure(figsize=(12, 12))
        ax = plt.imshow(self.images_matrix.T, cmap='gray', aspect='auto')

    def save_image_array(self, basename=None):
        if basename is None:
            basename = '{}_modified'.format(self.filename.split('.')[0])
        np.save('{}.npy'.format(basename), self.images_matrix)


In [2]:
md = MotionDetector('pumpjack.mp4')
md.image_directory_name = 'images'
md.display_original_movie()

In [3]:
#Display grayscale movie
md.create_movie()
md.display_movie()

ffmpeg version 4.3.6-0+deb11u1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 

In [4]:
md.create_image_difference(rank=1)
md.create_movie()
md.display_movie()

(227, 36576)


ffmpeg version 4.3.6-0+deb11u1 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 10 (Debian 10.2.1-6)
  configuration: --prefix=/usr --extra-version=0+deb11u1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab 